In [ ]:
import backtrader as bt
import pandas as pd
import yfinance as yf
import numpy as np

from utils.tester import log_results, test_strategy, load_data
from utils.strategies import TriplleMARSIFracStrategy

## Source data locally

In [ ]:
data_local = pd.read_csv("../data/xauusd_1h.csv", index_col=None)
data_local['datetime'] = pd.to_datetime(data_local['datetime'])
print(data_local.head())

data_feed_local = bt.feeds.PandasData(dataname=data_local, name="xau",
                                      datetime=0, openinterest=-1,
                                      open=1,
                                      high=2,
                                      low=3,
                                      close=4,
                                      volume=5)

data_feeds_local = dict(xau=data_feed_local)

## Download Data from YF

In [ ]:
# Download SPX500 data from Yahoo Finance
yf_params = {'start': '2018-06-01', 'end': '2025-01-01',
             'interval': '1d', 'multi_level_index': False, 'auto_adjust': True}
data_names = [
    # '^RUT', '^IXIC', '^VIX',
    # '^DJI', '^GSPC', 'GOOG', 'MSFT', 'AMZN', 'TSLA', 
            #   'NFLX', 'NVDA', 'AMD', 'KO',
              'BTC-USD', 'ETH-USD', 'BNB-USD', 'DOGE-USD', 'SOL-USD'
              ]

feeds = load_data(data_names, yf_params)

# Start Backtesting

In [ ]:
strategy_params = dict(
                       atr_period=11,
                       atr_multiplier=1,
                       rrr=2.5,
)
cerebro_params = dict(percents=2)
results = dict()

for name, feed in feeds.items():
    results[name], cereb = test_strategy(TriplleMARSIFracStrategy, feed, strategy_params, cerebro_params)
    # cereb.plot(iplot=True)

In [ ]:
for name, res in results.items():
    log_results(name, res)